# Web Scaping

## Scraping a single page

In [4]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser 
import textstat

In [5]:
#This is just to make it so the request goes through and the Dawson website does not decline it 
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'referrer': 'https://google.com'
}

In [6]:
# URL of program page for pure and applied
url = 'https://www.dawsoncollege.qc.ca/pure-applied/'
r = requests.get(url, headers=headers)

In [8]:
# Status code is 200 means all is good and request went through
r.status_code

200

In [10]:
# Get HTML of the page
html = r.text.strip()

In [13]:
# Make html of page a BeautifulSoup 
soup = BeautifulSoup(html, 'lxml')

In [24]:
# Header Content of page
content = soup.find(class_='entry-content')
#Get program title and strip any white spaces
programTitle = content.find('h2').contents[0].strip()
print(programTitle)

Pure & Applied Science (200.P2)
